In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Glioblastoma"
cohort = "GSE129978"

# Input paths
in_trait_dir = "../../input/GEO/Glioblastoma"
in_cohort_dir = "../../input/GEO/Glioblastoma/GSE129978"

# Output paths
out_data_file = "../../output/preprocess/Glioblastoma/GSE129978.csv"
out_gene_data_file = "../../output/preprocess/Glioblastoma/gene_data/GSE129978.csv"
out_clinical_data_file = "../../output/preprocess/Glioblastoma/clinical_data/GSE129978.csv"
json_path = "../../output/preprocess/Glioblastoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"AHR activation in glioblastoma cells and gene expression analysis of T cells in TCL1-AT mice."
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['strain: C56BL/6'], 1: ['gender: female']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
from typing import Optional

# 1. Evaluate gene expression availability
# Based on the background information, this appears to be gene expression data from GBM cells
is_gene_available = True

# 2. Analyze variable availability and create conversion functions

# 2.1 Find keys for trait, age, and gender in sample characteristics dictionary
# From the sample characteristics, we see cell lines U-87MG and CAS-1, which are glioblastoma cell lines
trait_row = 0  # Cell line information is in the first row
age_row = None  # No age information available for cell lines
gender_row = None  # No gender information available for cell lines

# 2.2 Create conversion functions
def convert_trait(value: str) -> Optional[int]:
    """Convert trait information to binary (0: non-glioblastoma, 1: glioblastoma)"""
    if pd.isna(value):
        return None
    
    # Handle string or other type inputs
    value_str = str(value)
        
    # Extract the value after colon if present
    if ':' in value_str:
        value_str = value_str.split(':', 1)[1].strip()
    
    # U-87MG and CAS-1 are both glioblastoma cell lines
    if "U-87MG" in value_str or "CAS-1" in value_str:
        return 1
    return None

def convert_age(value: str) -> Optional[float]:
    """Convert age information to continuous numeric value"""
    # Not used in this dataset, but implementing for completeness
    if pd.isna(value):
        return None
    
    # Handle string or other type inputs
    value_str = str(value)
        
    if ':' in value_str:
        value_str = value_str.split(':', 1)[1].strip()
    
    try:
        return float(value_str)
    except:
        return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender information to binary (0: female, 1: male)"""
    # Not used in this dataset, but implementing for completeness
    if pd.isna(value):
        return None
    
    # Handle string or other type inputs
    value_str = str(value)
        
    if ':' in value_str:
        value_str = value_str.split(':', 1)[1].strip().lower()
    
    if value_str in ['female', 'f']:
        return 0
    elif value_str in ['male', 'm']:
        return 1
    return None

# 3. Save metadata about dataset usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if available
if is_trait_available:
    # Create clinical_data DataFrame from the sample characteristics dictionary provided
    sample_chars_dict = {
        0: ['cell line: U-87MG', 'cell line: CAS-1'], 
        1: ['experiment: FICZ_KynA', 'experiment: I3CA', 'experiment: I3P_HPP_PP', 
            'experiment: IL4I1_shAHR', 'experiment: IL4I1_KO'], 
        2: ['agent: DMSO', 'agent: FICZ 100nm', 'agent: KynA 50uM', 'agent: I3CA 50uM', 
            'agent: HPP 40uM', 'agent: I3P 40uM', 'agent: PP 40uM', 'condition: shC_shAHR1', 
            'condition: shC_shAHR2', 'condition: shC', 'condition: IL4I1_OE_shAHR1', 
            'condition: IL4I1_OE_shAHR2', 'condition: IL4I1_OE_shC', 'condition: IL4I1_KO', 
            'condition: NTC', 'condition: NTC.01'], 
        3: ['shRNA: shCtrl', np.nan]
    }
    
    # Format the clinical data to meet the expectations of geo_select_clinical_features
    clinical_data = pd.DataFrame()
    for row_idx, values in sample_chars_dict.items():
        row_data = pd.DataFrame({'row_idx': row_idx, 'value': values})
        clinical_data = pd.concat([clinical_data, row_data], ignore_index=True)
    
    # Apply the geo_select_clinical_features function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    # Preview the resulting dataframe
    preview = preview_df(selected_clinical_df)
    print(f"Preview of clinical data:\n{preview}")
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data:
{'row_idx': [nan], 'value': [1.0]}
Clinical data saved to ../../output/preprocess/Glioblastoma/clinical_data/GSE129978.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 68
Header line: "ID_REF"	"GSM4514184"	"GSM4514185"	"GSM4514186"	"GSM4514187"	"GSM4514188"	"GSM4514189"	"GSM4514190"	"GSM4514191"	"GSM4514192"	"GSM4514193"	"GSM4514194"	"GSM4514195"	"GSM4514196"	"GSM4514197"	"GSM4514198"	"GSM4514199"
First data line: 17200001	6.0912	5.3400	5.7945	3.3472	5.7467	5.6059	5.4014	5.8888	5.3955	5.7953	4.3194	3.9623	5.0397	5.5958	4.8685	3.7608
Index(['17200001', '17200003', '17200005', '17200007', '17200009', '17200011',
       '17200013', '17200015', '17200017', '17200019', '17200021', '17200023',
       '17200025', '17200027', '17200029', '17200031', '17200033', '17200035',
       '17200037', '17200039'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers appear to be numeric IDs (like "16657439")
# These are not standard human gene symbols, which would typically be names like "BRCA1", "TP53", etc.
# These look like probe IDs or transcript IDs that would need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE129978
Line 6: !Series_title = AHR activation in glioblastoma cells and gene expression analysis of T cells in TCL1-AT mice.
Line 7: !Series_geo_accession = GSE129978
Line 8: !Series_status = Public on Aug 19 2020
Line 9: !Series_submission_date = Apr 17 2019
Line 10: !Series_last_update_date = Nov 18 2020
Line 11: !Series_pubmed_id = 32818467
Line 12: !Series_summary = This SuperSeries is composed of the SubSeries listed below.
Line 13: !Series_overall_design = Refer to individual Series
Line 14: !Series_type = Expression profiling by array
Line 15: !Series_sample_id = GSM3728779
Line 16: !Series_sample_id = GSM3728780
Line 17: !Series_sample_id = GSM3728781
Line 18: !Series_sample_id = GSM3728782
Line 


Gene annotation preview:
{'ID': ['17210850', '17210852', '17210855', '17210869', '17210883'], 'probeset_id': [17210850.0, 17210852.0, 17210855.0, 17210869.0, 17210883.0], 'SPOT_ID': ['chr1(+):3102016-3102125', 'chr1(+):3466587-3513553', 'chr1(+):4807823-4846739', 'chr1(+):4857694-4897909', 'chr1(+):4970857-4976820'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': [3102016.0, 3466587.0, 4807823.0, 4857694.0, 4970857.0], 'stop': [3102125.0, 3513553.0, 4846739.0, 4897909.0, 4976820.0], 'total_probes': [8.0, 23.0, 20.0, 21.0, 23.0], 'gene_assignment': ['---', '---', 'NM_008866 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// ENSMUST00000027036 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// BC013536 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// BC052848 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// U89352 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// CT010201 // Lypla1 // lysophospholipase 1 /

### Step 6: Gene Identifier Mapping

In [7]:
# Step 1: Let's examine the SOFT file more thoroughly to identify the correct platform
import gzip
import re

# Look for platform information in the SOFT file
platform_id = None
with gzip.open(soft_file, 'rt') as file:
    for line in file:
        if line.startswith('!Series_platform_id'):
            platform_id = line.strip().split('=')[1].strip()
            print(f"Platform ID found: {platform_id}")
            break

# Extract platform-specific annotation data
platform_data = []
in_platform_section = False
with gzip.open(soft_file, 'rt') as file:
    for line in file:
        if line.startswith(f'^PLATFORM = {platform_id}'):
            in_platform_section = True
            continue
        elif line.startswith('^') and in_platform_section:
            # We've moved to another section
            break
        elif in_platform_section and line.startswith('!platform_table_begin'):
            # Next line should be the header
            header = next(file).strip()
            platform_data.append(header)
            # Read until the end of the platform table
            for table_line in file:
                if table_line.startswith('!platform_table_end'):
                    break
                platform_data.append(table_line.strip())
            break

# If we found platform data, convert it to a DataFrame
if platform_data:
    import pandas as pd
    import io
    platform_text = '\n'.join(platform_data)
    platform_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                     low_memory=False, on_bad_lines='skip')
    print("\nPlatform annotation preview:")
    print(preview_df(platform_annotation))
    
    # Extract ID and gene columns
    id_col = 'ID' if 'ID' in platform_annotation.columns else platform_annotation.columns[0]
    
    # Check potential gene name columns
    potential_gene_cols = [col for col in platform_annotation.columns 
                          if any(term in col.lower() for term in ['gene', 'symbol', 'mrna'])]
    print(f"\nPotential gene name columns: {potential_gene_cols}")
    
    if potential_gene_cols:
        gene_col = potential_gene_cols[0]
        # Create mapping dataframe
        mapping_df = platform_annotation[[id_col, gene_col]].copy()
        mapping_df = mapping_df.rename(columns={id_col: 'ID', gene_col: 'Gene'})
        mapping_df['ID'] = mapping_df['ID'].astype(str)
        mapping_df = mapping_df.dropna()
        
        # Check for gene symbols
        is_gene_symbols = True
        sample_genes = mapping_df['Gene'].dropna().head(10).tolist()
        if len(sample_genes) > 0:
            print(f"\nSample genes: {sample_genes}")
            # Check if we need to extract symbols
            if any('/' in str(gene) or '//' in str(gene) for gene in sample_genes):
                is_gene_symbols = False
        
        # If not direct gene symbols, extract them
        if not is_gene_symbols:
            # Use the mouse gene extraction function
            def extract_mouse_gene_symbols(text):
                """Extract mouse gene symbols from gene_assignment text"""
                if not isinstance(text, str):
                    return []
                
                # Extract gene symbols after "//" pattern
                symbols = []
                matches = re.findall(r'[A-Z]+\d+_\d+ // ([A-Za-z0-9-]+) //', text)
                if matches:
                    symbols.extend(matches)
                
                # If no matches found, try a more general pattern
                if not symbols:
                    matches = re.findall(r'// ([A-Za-z0-9][A-Za-z0-9-]{1,20}) //', text)
                    if matches:
                        symbols.extend(matches)
                
                return list(dict.fromkeys(symbols))
            
            mapping_df['Gene'] = mapping_df['Gene'].apply(extract_mouse_gene_symbols)
            mapping_df = mapping_df.explode('Gene').dropna(subset=['Gene'])
    else:
        # If no obvious gene column, try to extract from description or another field
        desc_col = next((col for col in platform_annotation.columns 
                        if any(term in col.lower() for term in ['desc', 'definition', 'assignment'])), None)
        
        if desc_col:
            mapping_df = platform_annotation[[id_col, desc_col]].copy()
            mapping_df = mapping_df.rename(columns={id_col: 'ID', desc_col: 'Gene'})
            mapping_df['ID'] = mapping_df['ID'].astype(str)
            mapping_df = mapping_df.dropna()
            
            # Extract gene symbols
            mapping_df['Gene'] = mapping_df['Gene'].apply(extract_human_gene_symbols)
            # Explode the list of genes into separate rows
            mapping_df = mapping_df.explode('Gene').dropna(subset=['Gene'])
        else:
            # If we can't find any gene annotation, we'll have to skip the mapping
            mapping_df = None

# Check if any IDs in mapping_df match gene_data index
if mapping_df is not None:
    common_ids = set(mapping_df['ID']).intersection(set(gene_data.index))
    print(f"\nNumber of common IDs between mapping and expression data: {len(common_ids)}")
    print(f"Sample of common IDs: {list(common_ids)[:5]}")
    
    # Apply gene mapping only if there are common IDs
    if common_ids:
        # Drop Gene column if it already exists
        if 'Gene' in gene_data.columns:
            gene_data = gene_data.drop(columns=['Gene'])
            
        # Apply the mapping
        gene_data = apply_gene_mapping(gene_data, mapping_df)
        
        # Preview the gene expression data
        print(f"\nGene expression data shape: {gene_data.shape}")
        print(f"First few gene symbols: {gene_data.index[:10].tolist()}")
    else:
        print("\nNo matching probe IDs found between annotation and expression data.")
        # In this case, we can't map to gene symbols, so we'll use probe IDs as is
        gene_data = gene_data.copy()
        print(f"Using probe IDs directly. Shape: {gene_data.shape}")
else:
    print("\nCould not find suitable gene annotation data")
    # Use probe IDs directly
    gene_data = gene_data.copy()

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Platform ID found: GPL16570



Platform annotation preview:
{'ID': ['17210850', '17210852', '17210855', '17210869', '17210883'], 'probeset_id': [17210850.0, 17210852.0, 17210855.0, 17210869.0, 17210883.0], 'SPOT_ID': ['chr1(+):3102016-3102125', 'chr1(+):3466587-3513553', 'chr1(+):4807823-4846739', 'chr1(+):4857694-4897909', 'chr1(+):4970857-4976820'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': [3102016.0, 3466587.0, 4807823.0, 4857694.0, 4970857.0], 'stop': [3102125.0, 3513553.0, 4846739.0, 4897909.0, 4976820.0], 'total_probes': [8.0, 23.0, 20.0, 21.0, 23.0], 'gene_assignment': ['---', '---', 'NM_008866 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// ENSMUST00000027036 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// BC013536 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// BC052848 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// U89352 // Lypla1 // lysophospholipase 1 // 1 A1|1 // 18777 /// CT010201 // Lypla1 // lysophospholipase


Number of common IDs between mapping and expression data: 27036
Sample of common IDs: ['17308796', '17269769', '17333375', '17520146', '17325874']

Gene expression data shape: (412, 16)
First few gene symbols: ['A530046M15', 'A530065N20', 'A630020A06', 'A730015I17', 'A730034C02', 'A730043L09', 'A830025M08', 'A830049A06', 'A830060N17', 'AA066038']
Gene expression data saved to ../../output/preprocess/Glioblastoma/gene_data/GSE129978.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Examine gene_data to decide on the right approach
# We need to check if we're dealing with gene symbols or just probe IDs
has_gene_symbols = False
if isinstance(gene_data.index[0], str) and any(c.isalpha() for c in gene_data.index[0]):
    # Check if index contains alphabetic characters, suggesting gene symbols
    has_gene_symbols = True

# Only normalize if we have gene symbols
if has_gene_symbols:
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")
else:
    # If no gene symbols, just use probe IDs directly
    normalized_gene_data = gene_data.copy()
    print(f"Using probe IDs directly. Gene data shape: {normalized_gene_data.shape}")
    print(f"Sample probe IDs: {list(normalized_gene_data.index[:10])}")

# Save the gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# 2. Generate clinical data - all samples are glioblastoma cell lines
# Create a proper clinical dataframe with trait information
clinical_df = pd.DataFrame(index=[trait], 
                          data={col: 1 for col in normalized_gene_data.columns})
print(f"Clinical data shape: {clinical_df.shape}")
print("Clinical data preview:")
print(clinical_df.iloc[:, :5])  # Show first 5 columns only

# Save the clinical features
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 5 columns):")
if linked_data.shape[1] >= 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data.head())

# 4. Handle missing values
print("\nMissing values before handling:")
print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
gene_cols = [col for col in linked_data.columns if col != trait]
if gene_cols:
    missing_genes_pct = linked_data[gene_cols].isna().mean()
    genes_with_high_missing = sum(missing_genes_pct > 0.2)
    print(f"  Genes with >20% missing: {genes_with_high_missing}")
    
    if len(linked_data) > 0:  # Ensure we have samples before checking
        missing_per_sample = linked_data[gene_cols].isna().mean(axis=1)
        samples_with_high_missing = sum(missing_per_sample > 0.05)
        print(f"  Samples with >5% missing genes: {samples_with_high_missing}")

# Only proceed with missing value handling if we have gene data
if len(gene_cols) > 0:
    cleaned_data = handle_missing_values(linked_data, trait)
    print(f"Data shape after handling missing values: {cleaned_data.shape}")
else:
    # If we only have trait data, we can't do much analysis
    cleaned_data = linked_data.copy()
    print("No gene data available for missing value handling.")

# 5. Evaluate bias in trait and demographic features
is_trait_biased = False
if len(cleaned_data) > 0:
    # If all samples have same trait value, it's biased
    if cleaned_data[trait].nunique() <= 1:
        is_trait_biased = True
        print(f"The feature '{trait}' is biased - all samples have the same value.")
    else:
        trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
        is_trait_biased = trait_biased
else:
    print("No data remains after handling missing values.")
    is_trait_biased = True

# 6. Final validation and save
note = "Dataset contains gene expression data from glioblastoma cell lines. "
note += "No demographic features available. " 
if len(gene_cols) == 0:
    note += "No usable gene measurements after processing."

is_gene_available = len(normalized_gene_data) > 0 and len(gene_cols) > 0
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=cleaned_data,
    note=note
)

# 7. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")

Gene data shape after normalization: (33, 16)
Sample gene symbols after normalization: ['ATP6', 'ATP8', 'C2', 'C3', 'C6', 'C7', 'C9', 'COX1', 'COX2', 'COX3']
Gene data saved to ../../output/preprocess/Glioblastoma/gene_data/GSE129978.csv
Clinical data shape: (1, 16)
Clinical data preview:
              GSM4514184  GSM4514185  GSM4514186  GSM4514187  GSM4514188
Glioblastoma           1           1           1           1           1
Clinical data saved to ../../output/preprocess/Glioblastoma/clinical_data/GSE129978.csv
Linked data shape: (16, 34)
Linked data preview (first 5 rows, first 5 columns):
            Glioblastoma     ATP6     ATP8      C2      C3
GSM4514184           1.0  10.7716  10.7716  3.2225  4.6422
GSM4514185           1.0  10.6608  10.6608  3.1908  5.4308
GSM4514186           1.0  10.6187  10.6187  3.0901  4.9176
GSM4514187           1.0  11.1555  11.1555  2.5386  3.9111
GSM4514188           1.0  10.8219  10.8219  3.1858  5.7965

Missing values before handling:
  Trait 